In [0]:
#!tar -xzvf  /content/drive/My\ Drive/LC82121112013339LGN00.tar
training_set_path='/content/drive/My Drive/Metadata/train.txt'
val_set_path='/content/drive/My Drive/Metadata/val.txt'
test_set_path='/content/drive/My Drive/Metadata/test.txt'
model_path='/content/drive/My Drive/models/'

num_classes=2
image_shape=(512,512,9)
padding=((0,0),(0,0))
batch_size=5 # num of chunks
epochs=10

train_subset_size = 2000 
val_subset_size = 500
n_splits = 1
set_info={ 
          2:[0,1,2,3,4,5,6,8,9]}
          

training_set_size=2000
val_set_size=500
test_set_size=899
optimizer='sgd' # can try adam .... prop
loss_function='categorical_crossentropy'
metrics=['accuracy']  # can add more metrics


In [0]:
from keras.layers import Activation,BatchNormalization,Conv2D
from keras.engine.topology import Layer
#import keras.backend as K
#import tensorflow as tf
import keras.backend.tensorflow_backend as K

class MaxPoolingWithIndices(Layer):
    def __init__(self, pool_size,strides,padding='SAME',**kwargs):
        super(MaxPoolingWithIndices, self).__init__(**kwargs)
        self.pool_size=pool_size
        self.strides=strides
        self.padding=padding
        return
    def call(self,x):
        pool_size=self.pool_size
        strides=self.strides
        if isinstance(pool_size,int):
            ps=[1,pool_size,pool_size,1]
        else:
            ps=[1,pool_size[0],pool_size[1],1]
        if isinstance(strides,int):
            st=[1,strides,strides,1]
        else:
            st=[1,strides[0],strides[1],1]
        output1,output2=K.tf.nn.max_pool_with_argmax(x,ps,st,self.padding)
        return [output1,output2]
    def compute_output_shape(self, input_shape):
        if isinstance(self.pool_size,int):
            output_shape=(input_shape[0],input_shape[1]//self.pool_size,input_shape[2]//self.pool_size,input_shape[3])
        else:
            output_shape=(input_shape[0],input_shape[1]//self.pool_size[0],input_shape[2]//self.pool_size[1],input_shape[3])
        return [output_shape,output_shape]


class UpSamplingWithIndices(Layer):
    def __init__(self, **kwargs):
        super(UpSamplingWithIndices, self).__init__(**kwargs)
        return
    def call(self,x):
        argmax=K.cast(K.flatten(x[1]),'int32')
        max_value=K.flatten(x[0])
        with K.tf.variable_scope(self.name):
            input_shape=K.shape(x[0])
            batch_size=input_shape[0]
            image_size=input_shape[1]*input_shape[2]*input_shape[3]
            output_shape=[input_shape[0],input_shape[1]*2,input_shape[2]*2,input_shape[3]]
            indices_0=K.flatten(K.tf.matmul(K.reshape(K.tf.range(batch_size),(batch_size,1)),K.ones((1,image_size),dtype='int32')))
            indices_1=argmax%(image_size*4)//(output_shape[2]*output_shape[3])
            indices_2=argmax%(output_shape[2]*output_shape[3])//output_shape[3]
            indices_3=argmax%output_shape[3]
            indices=K.tf.stack([indices_0,indices_1,indices_2,indices_3])
            output=K.tf.scatter_nd(K.transpose(indices),max_value,output_shape)
            return output
    def compute_output_shape(self, input_shape):
        return input_shape[0][0],input_shape[0][1]*2,input_shape[0][2]*2,input_shape[0][3]

def CompositeConv(inputs,num_layers,num_features):
    output=inputs
    if isinstance(num_features,int):
        for i in range(num_layers):
            output=Conv2D(num_features,(7,7),padding='same')(output)
            output=BatchNormalization(axis=3)(output)
            output=Activation('relu')(output)
        return output
    for i in range(num_layers):
        output=Conv2D(num_features[i],(7,7),padding='same')(output)
        output=BatchNormalization(axis=3)(output)
        output=Activation('relu')(output)
    return output


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.models import Model
from keras.layers import Activation,Input,ZeroPadding2D,Cropping2D


def create_model():
    inputs=Input(shape=image_shape)

    x = ZeroPadding2D(padding)(inputs)

    x=CompositeConv(x,2,64)
    x,argmax1=MaxPoolingWithIndices(pool_size=2,strides=2)(x)
    
    x=CompositeConv(x,2,64)
    x,argmax2=MaxPoolingWithIndices(pool_size=2,strides=2)(x)
    
    x=CompositeConv(x,3,64)
    x,argmax3=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=CompositeConv(x,3,64)
    x,argmax4=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=CompositeConv(x,3,64)
    x,argmax5=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=UpSamplingWithIndices()([x,argmax5])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax4])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax3])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax2])
    x=CompositeConv(x,2,64)
    
    x=UpSamplingWithIndices()([x,argmax1])
    x=CompositeConv(x,2,[64,num_classes])

    x=Activation('softmax')(x)

    y=Cropping2D(padding)(x)
    my_model=Model(inputs=inputs,outputs=y)
    
    return my_model

In [0]:
import imageio
import numpy as np
from keras.utils import to_categorical


def read_image_batch(image_list, batch_size, channel_list):
    while True:
        l=len(image_list)
        num_batch=l//batch_size
        if num_batch*batch_size<l:
            num_batch+=1
        for i in range(num_batch):
           
            batch_set=image_list[batch_size*i:min(batch_size*(i+1),l),:]
            batch_set=[batch_set[bs] for bs in range(len(batch_set))]
            X=np.array([np.load(line[0][0:]) for line in batch_set])
            labels=np.array([np.load(line[1][0:]) for line in batch_set])
            y=to_categorical(labels,num_classes)
            X = X[:, :, :, channel_list]
            yield tuple((X, y))


In [0]:
test_names=[]

with open(test_set_path,"r") as f:
  test_image_list=[]
  for line in f.readlines():
    arr=[]
    str_array=line.split(" ")
    arr.append(str_array[0]+" "+str_array[1])
    arr.append(str_array[2]+" "+str_array[3][:-1])
    test_image_list.append(arr)
    test_names.append(line)

test_image_list=np.asarray(test_image_list)


In [0]:
import imageio
import numpy as np
from keras.utils import to_categorical

'''
def read_test_image_batch(data_path,batch_size,channel):
  image_list=[]
  with open(data_path,"r") as f:
    for line in f.readlines():
      arr=[]
      str_array=line.split(" ")
      arr.append(str_array[0]+" "+str_array[1])
      arr.append(str_array[2]+" "+str_array[3][:-1])
      image_list.append(arr)
  image_list=np.asarray(image_list)
 
  
  while True: 
    l=len(image_list)
    num_batch=l//batch_size
    if num_batch*batch_size<l:
        num_batch+=1
       
    for i in range(num_batch):
      batch_set=image_list[batch_size*i:min(batch_size*(i+1),l),:]
      batch_set=[batch_set[bs] for bs in range(len(batch_set))]
      X=np.array([np.load(line[0][0:]) for line in batch_set])
      X=X[:,:,:,channel]
      labels=np.array([np.load(line[1][0:]) for line in batch_set])
      y=to_categorical(labels,num_classes)
      
      yield tuple((X, y))
'''

def read_test_image_batch(image_list, batch_size, channel_list):
    while True:
        l = len(image_list)
        num_batch = l // batch_size
        if num_batch*batch_size<l:
            num_batch+=1
        for i in range(num_batch):
            batch_set=image_list[batch_size*i:min(batch_size*(i+1),l),:]
            batch_set=[batch_set[bs] for bs in range(len(batch_set))]
            X=np.array([np.load(line[0][0:]) for line in batch_set])
            labels=np.array([np.load(line[1][0:]) for line in batch_set])
            y=to_categorical(labels,num_classes)
            X = X[:, :, :, channel_list]
            yield tuple((X, y))
  





In [0]:
import argparse
from keras.models import Model
from keras.layers import Activation,Input
from scipy.stats import mode
import os

import argparse
from keras.models import Model
from keras.layers import Activation,Input
from scipy.stats import mode
import os
import time
#from custom_layers import MaxPoolingWithIndices,UpSamplingWithIndices,CompositeConv
#import config as cf
#from util import read_image_batch
#from create_model import create_model
tic = time.time()
print("start time: {}".format(tic))
class_labels=[]
model_name = "model_rich_feature_0"
current_model = os.path.join(model_path, model_name)
assert os.path.exists(current_model)
toc = time.time()

test_data=read_test_image_batch(test_image_list,batch_size,set_info[2])

toc = time.time() - toc
print("test_data loaded in {1:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

my_model=create_model()
toc = time.time() - toc
print("model created in {1:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

my_model.compile(optimizer,loss=loss_function,metrics=metrics)
toc = time.time() - toc
print("model compiled in {1:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

my_model.load_weights(current_model)
toc = time.time() - toc
print("weights loaded in {1:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))


probs=my_model.predict(test_data,steps=(test_set_size+1)//batch_size)
toc = time.time() - toc
print("predictions generated in {1:.2f} seconds\ntotal runtime: {1:.2f}".format(toc, time.time() - tic))

class_labels.append(probs.argmax(axis=-1))


start time: 1573680193.172696
test_data loaded in 0.00 seconds
total runtime: 0.00
model created in 3.09 seconds
total runtime: 3.09
model compiled in 3.13 seconds
total runtime: 3.13
weights loaded in 11.98 seconds
total runtime: 11.98
predictions generated in 146.93 seconds
total runtime: 146.93


In [0]:
import numpy as np

In [0]:
output = class_labels[0]


In [0]:
print(output.shape)
hist = np.histogram(output, bins=3)

(899, 512, 512)


In [0]:
print(hist)

(array([214368380,         0,  21299076]), array([0.        , 0.33333333, 0.66666667, 1.        ]))


In [0]:
test_scene_dir = "/content/drive/My Drive/test_chunks"
output_dir = "/content/drive/My Drive/output"

if not os.path.exists(os.path.join(output_dir, model_name)):
  os.mkdir(os.path.join(output_dir, model_name))

test_scene_ids = [i for i in os.listdir(test_scene_dir) if os.path.isdir(os.path.join(test_scene_dir, i))]
print(test_scene_ids)
for i in test_scene_ids:
  output_scene_dir = os.path.join(output_dir, model_name, i)
  if not os.path.exists(output_scene_dir):
    os.mkdir(output_scene_dir)
    print("output_dir made at {}".format(output_scene_dir))

['LC82201072015017LGN00', 'LC82071182013336LGN00', 'LC81431082013352LGN00', 'LC81681112014002LGN00', 'LC80631112014002LGN00']
output_dir made at /content/drive/My Drive/output/model_rich_feature_0/LC82201072015017LGN00
output_dir made at /content/drive/My Drive/output/model_rich_feature_0/LC82071182013336LGN00
output_dir made at /content/drive/My Drive/output/model_rich_feature_0/LC81431082013352LGN00
output_dir made at /content/drive/My Drive/output/model_rich_feature_0/LC81681112014002LGN00
output_dir made at /content/drive/My Drive/output/model_rich_feature_0/LC80631112014002LGN00


In [0]:
class_labels=np.asarray(class_labels)
print(class_labels.shape)

u,indices=np.unique(class_labels,return_inverse=True)
final_labels=u[np.argmax(np.apply_along_axis(np.bincount,0,indices.reshape(class_labels.shape),None,np.max(indices)+1),axis=0)]



(1, 899, 512, 512)


In [0]:
print("final labels")
print(final_labels.shape)

for i in range(test_set_size):
    line=test_names[i]
    print(line)
    str_array=line.split(" ")
    scene_id=str_array[1][18:40]
    out_str=str_array[1][40:-4]+"_output.npy"
    print(out_str)
    print("\nsaved to:")
    print(os.path.join(output_dir, model_name, scene_id, out_str))
    np.save(os.path.join(output_dir, model_name, scene_id, out_str),final_labels[i])

In [0]:
import argparse
from keras.utils import multi_gpu_model
from keras.models import Model
from keras.layers import Activation,Input



def getSubsets(data_path, set_info, subset_size):
  image_list=[]
  with open(data_path,"r") as f:
    for line in f.readlines():
      arr=[]
      str_array=line.split(" ")
      arr.append(str_array[0]+" "+str_array[1])
      arr.append(str_array[2]+" "+str_array[3][:-1])
      image_list.append(arr)

    image_list = np.asarray(image_list)
    np.random.seed(1)
    np.random.shuffle(image_list)
    data = []
    jump = int(subset_size - ((subset_size*n_splits)-image_list.shape[0])/2)
    for i in range(0, image_list.shape[0]-subset_size, jump):
      #arr = np.random.randint(0, len(image_list), subset_size)
      #subset = image_list[arr,]
      subset = image_list[i: i+subset_size,]
      for key, value in set_info.items():        
          data.append(read_image_batch(subset, batch_size, value))
           
  return np.asarray(data)
            

def main(args):
    #make_segnet_input_file('/content/drive/My Drive',['LC82121112013339LGN00'],'/content/drive/My Drive/Metadata/train.txt')
    
    train_data = getSubsets(training_set_path, set_info, train_subset_size)
    val_data = getSubsets(val_set_path, set_info, val_subset_size)
  
    key=1
    for i in range(1, len(train_data)):
      my_model=create_model()
      my_model.compile(optimizer,loss=loss_function,metrics=metrics)
      if args['resume']:
        my_model.load_weights(model_path+args['load']+"_"+str(i)+"_key_"+str(key))
      min_loss = 1.0
      for j in range(epochs):
        hist = my_model.fit_generator(train_data[i],
                                steps_per_epoch=(train_subset_size+1)//batch_size,
                                epochs=1,validation_data=val_data[i],
                                validation_steps=(val_subset_size+1)//batch_size)
        val_loss = hist.history["val_loss"]
        print(val_loss)
        if val_loss[0] < min_loss:
          min_loss = val_loss[0]
          my_model.save_weights(model_path+args['save']+"_"+str(i)+"_key_"+str(key))
        print("model fit epoch: "+str(j))
        #my_model.save_weights(model_path+args['save']+"_"+str(i)+"_key_"+str(key))
     
      
    
#parser = argparse.ArgumentParser()
#parser.add_argument("--save",default='my_model')
#parser.add_argument("--resume",action='store_true')
#parser.add_argument("--load",default='my_model')
args = {"save": "model_B", "resume":False, "load": "model_B"}
# main(args)


In [0]:
import argparse
from keras.models import Model
from keras.layers import Activation,Input
from scipy.stats import mode
import os
#from custom_layers import MaxPoolingWithIndices,UpSamplingWithIndices,CompositeConv
#import config as cf
#from util import read_image_batch
#from create_model import create_model

class_labels=[]


for i in os.listdir(model_path):
  if(i==".ipynb_checkpoints"):
    continue
  print(model_path+i)
  my_model=create_model()
  my_model.compile(optimizer,loss=loss_function,metrics=metrics)
  my_model.load_weights(model_path+i)
  key=int(i[-1:])
  if(key!=0):
    continue
  test_data=read_test_image_batch(test_set_path,batch_size,set_info[key])
  
  probs=my_model.predict(test_data,steps=(test_set_size+1)//batch_size)
  
  class_labels.append(probs.argmax(axis=-1))
  
  
class_labels=np.asarray(class_labels)
print(class_labels.shape)



u,indices=np.unique(class_labels,return_inverse=True)
final_labels=u[np.argmax(np.apply_along_axis(np.bincount,0,indices.reshape(class_labels.shape),None,np.max(indices)+1),axis=0)]

print("final labels")
print(final_labels.shape)
#print("\n\n")
#print(final_labels)


for i in range(test_set_size):
  line=test_names[i]
  print(line)
  str_array=line.split(" ")
  scene_id=str_array[1][18:40]
  out_str=str_array[1][40:-4]+"_output.npy"
  print(out_str)
  np.save('/content/drive/My Drive/new_test_outputs_key_1/'+scene_id+out_str,final_labels[i])
  


/content/drive/My Drive/models/model_B_2_key_1
/content/drive/My Drive/models/model_rich_feature_0


ValueError: ignored

In [0]:
import os


"""
This script takes a list of scene ids and creates a file that can be used as input for a segnet model
@param string chunk_dir: The abspath base directory where each set of chunks for a scene has its own dir named with its sceneID
@param list scene_ids: A list of sceneIDs that exist in the chunk_dir. The chunks of these scenes will be used in the file.
@param string out_path: The abspath where the resulting file should be saved.
@return int lines_written: the total number of lines (corresponding to a data and label chunk path) in the file.
file format:
/path/to/scene_chunk.npy /path/to/scene_chunk_label.npy
/path/to/scene_chunk.npy /path/to/scene_chunk_label.npy
/path/to/scene_chunk.npy /path/to/scene_chunk_label.npy
...
"""

def make_segnet_input_file(chunk_dir, scene_ids, out_path):
    existing_scenes = [i for i in os.listdir(chunk_dir) if os.path.isdir(os.path.join(chunk_dir, i))]
    # filter out ids that don't exist in the given dir
    scene_ids = [i for i in scene_ids if i in existing_scenes]
    print(scene_ids)

    lines_to_write = []

    for i in scene_ids:
        scene_dir = os.path.join(chunk_dir, i)
        for j in os.listdir(scene_dir):
            if j[-9:] != "label.npy":
                data_path = os.path.join(scene_dir, j)
                file_split = os.path.splitext(j)
                label_path = os.path.join(scene_dir, file_split[0] + "_label" + file_split[1])

                lines_to_write.append("{} {}\n".format(data_path, label_path))

    with open(out_path, 'w+') as output:
        output.writelines(lines_to_write)

    return len(lines_to_write)